# Spark SQL Examples

Run the code cells below. This is the same code from the previous screencast.

In [1]:
import findspark
findspark.init('/home/yannick/dev/spark-2.4.4-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL") \
    .getOrCreate()

In [10]:
path = "./police-department-calls-for-service.json"
crime_log = spark.read.json(path, multiLine=True)

In [11]:
crime_log.take(1)

[Row(address='Geary Bl/divisadero St', address_type='Intersection', agency_id='1', call_date='2018-12-31T00:00:00.000', call_date_time='2018-12-31T23:57:00.000', call_time='23:57', city='San Francisco', common_location='', crime_id='183653763', disposition='ADM', offense_date='2018-12-31T00:00:00.000', original_crime_type_name='Traffic Stop', report_date='2018-12-31T00:00:00.000', state='CA')]

In [12]:
crime_log.printSchema()

root
 |-- address: string (nullable = true)
 |-- address_type: string (nullable = true)
 |-- agency_id: string (nullable = true)
 |-- call_date: string (nullable = true)
 |-- call_date_time: string (nullable = true)
 |-- call_time: string (nullable = true)
 |-- city: string (nullable = true)
 |-- common_location: string (nullable = true)
 |-- crime_id: string (nullable = true)
 |-- disposition: string (nullable = true)
 |-- offense_date: string (nullable = true)
 |-- original_crime_type_name: string (nullable = true)
 |-- report_date: string (nullable = true)
 |-- state: string (nullable = true)



# Create a View And Run Queries

The code below creates a temporary view against which you can run SQL queries.

In [13]:
crime_log.createOrReplaceTempView("crime_log_table")

In [14]:
spark.sql("SELECT * FROM crime_log_table LIMIT 2").show()

+--------------------+------------+---------+--------------------+--------------------+---------+-------------+---------------+---------+-----------+--------------------+------------------------+--------------------+-----+
|             address|address_type|agency_id|           call_date|      call_date_time|call_time|         city|common_location| crime_id|disposition|        offense_date|original_crime_type_name|         report_date|state|
+--------------------+------------+---------+--------------------+--------------------+---------+-------------+---------------+---------+-----------+--------------------+------------------------+--------------------+-----+
|Geary Bl/divisade...|Intersection|        1|2018-12-31T00:00:...|2018-12-31T23:57:...|    23:57|San Francisco|               |183653763|        ADM|2018-12-31T00:00:...|            Traffic Stop|2018-12-31T00:00:...|   CA|
|   100 Blk Howard St|Geo-Override|        1|2018-12-31T00:00:...|2018-12-31T23:54:...|    23:54|San Francis

In [15]:
spark.sql('''
          SELECT * 
          FROM crime_log_table 
          LIMIT 2
          '''
          ).show()

+--------------------+------------+---------+--------------------+--------------------+---------+-------------+---------------+---------+-----------+--------------------+------------------------+--------------------+-----+
|             address|address_type|agency_id|           call_date|      call_date_time|call_time|         city|common_location| crime_id|disposition|        offense_date|original_crime_type_name|         report_date|state|
+--------------------+------------+---------+--------------------+--------------------+---------+-------------+---------------+---------+-----------+--------------------+------------------------+--------------------+-----+
|Geary Bl/divisade...|Intersection|        1|2018-12-31T00:00:...|2018-12-31T23:57:...|    23:57|San Francisco|               |183653763|        ADM|2018-12-31T00:00:...|            Traffic Stop|2018-12-31T00:00:...|   CA|
|   100 Blk Howard St|Geo-Override|        1|2018-12-31T00:00:...|2018-12-31T23:54:...|    23:54|San Francis

In [16]:
spark.sql('''
          SELECT COUNT(*) 
          FROM crime_log_table 
          '''
          ).show()

+--------+
|count(1)|
+--------+
|  199999|
+--------+



In [17]:
spark.sql('''
          SELECT address, address_type, address_type, call_date
          FROM crime_log_table 
          WHERE crime_id == '183653763'
          '''
          ).collect()

[Row(address='Geary Bl/divisadero St', address_type='Intersection', address_type='Intersection', call_date='2018-12-31T00:00:00.000')]

In [20]:
spark.sql('''
          SELECT DISTINCT original_crime_type_name
          FROM crime_log_table 
          ORDER BY original_crime_type_name ASC
          '''
          ).show()

+------------------------+
|original_crime_type_name|
+------------------------+
|           "Bumper Jack"|
|                  $1,295|
|            $2,520 Worth|
|                    $200|
|                 & 415's|
|             'S Drinking|
|                  ***909|
|                 **909**|
|          **Disregard***|
|                   *909*|
|        *909* Encampment|
|                       .|
|                     .25|
|                 .25/.70|
|                 .25/500|
|                .25/500b|
|                .25/500e|
|                .25/500h|
|                .25/500l|
|                     .26|
+------------------------+
only showing top 20 rows



# User Defined Functions

In [23]:
spark.udf.register("get_hour", lambda x: int(datetime.datetime.fromtimestamp(x / 1000.0).hour))

<function __main__.<lambda>(x)>

In [25]:
spark.sql('''
          SELECT *, get_hour(call_time) AS hour
          FROM crime_log_table 
          LIMIT 1
          '''
          ).collect()

Py4JJavaError: An error occurred while calling o467.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 215, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/yannick/dev/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/yannick/dev/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/yannick/dev/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/yannick/dev/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/

In [17]:
songs_in_hour = spark.sql('''
          SELECT get_hour(ts) AS hour, COUNT(*) as plays_per_hour
          FROM user_log_table
          WHERE page = "NextSong"
          GROUP BY hour
          ORDER BY cast(hour as int) ASC
          '''
          )

In [24]:
songs_in_hour.show()

+----+--------------+
|hour|plays_per_hour|
+----+--------------+
|   0|           456|
|   1|           454|
|   2|           382|
|   3|           302|
|   4|           352|
|   5|           276|
|   6|           348|
|   7|           358|
|   8|           375|
|   9|           249|
|  10|           216|
|  11|           228|
|  12|           251|
|  13|           339|
|  14|           462|
|  15|           479|
|  16|           484|
|  17|           430|
|  18|           362|
|  19|           295|
+----+--------------+
only showing top 20 rows



# Converting Results to Pandas

In [18]:
songs_in_hour_pd = songs_in_hour.toPandas()

In [19]:
print(songs_in_hour_pd)

   hour  plays_per_hour
0     0             375
1     1             456
2     2             454
3     3             382
4     4             302
5     5             352
6     6             276
7     7             348
8     8             358
9     9             375
10   10             249
11   11             216
12   12             228
13   13             251
14   14             339
15   15             462
16   16             479
17   17             484
18   18             430
19   19             362
20   20             295
21   21             257
22   22             248
23   23             369
